First, we load data into memory and perform a simple statistical analysis on the data. And "data" stands for the train data, "oosdata" stands for the test data.

In [1]:
set.seed(666)
data <- load(url("https://github.com/zhentaoshi/Econ5821/raw/main/data_example/dataset_inf.Rdata"))
oosdata <- load(url("https://github.com/zhentaoshi/Econ5821/raw/main/data_example/data_oos.Rdata"))
summary(cpi)
summary(ppi)
summary(real.cpi)
summary(real.ppi)

     month             CPI       
 Min.   :  1.00   Min.   : 98.2  
 1st Qu.: 42.75   1st Qu.:101.5  
 Median : 84.50   Median :102.3  
 Mean   : 84.50   Mean   :102.7  
 3rd Qu.:126.25   3rd Qu.:103.6  
 Max.   :168.00   Max.   :108.7  

     month             PPI        
 Min.   :  1.00   Min.   : 91.80  
 1st Qu.: 42.75   1st Qu.: 97.92  
 Median : 84.50   Median :102.46  
 Mean   : 84.50   Mean   :101.46  
 3rd Qu.:126.25   3rd Qu.:105.22  
 Max.   :168.00   Max.   :110.10  

     month            CPI       
 Min.   :169.0   Min.   :100.8  
 1st Qu.:176.2   1st Qu.:101.5  
 Median :183.5   Median :101.8  
 Mean   :183.5   Mean   :101.9  
 3rd Qu.:190.8   3rd Qu.:102.3  
 Max.   :198.0   Max.   :102.9  

     month            PPI       
 Min.   :169.0   Min.   :100.0  
 1st Qu.:176.2   1st Qu.:102.8  
 Median :183.5   Median :104.2  
 Mean   :183.5   Mean   :104.0  
 3rd Qu.:190.8   3rd Qu.:105.7  
 Max.   :198.0   Max.   :107.8  

Then, we join the features, cpi and ppi together using \`month\`. 

In [2]:
data <- merge(X, cpi, by="month")
data <- merge(data, ppi, by="month")

oosdata <- merge(real.X, real.cpi, by="month")
oosdata <- merge(oosdata, real.ppi, by="month")

We will use r square as our criteria.

We first try XGBoost.

As XGBoost using specific data format, we must transform our data to DMatrix.

In [3]:
library(xgboost)
library(Matrix)

train_data = data
test_data = oosdata
trainX = data.matrix(train_data[,c(2:152,154)])
trainX = Matrix(trainX,sparse=T)
trainY = train_data[,153]
data = list(x = trainX, y = trainY )
dtrain = xgb.DMatrix(data = data$x, label = data$y)

testX = data.matrix(test_data[,c(2:152,154)])
testX = Matrix(testX,sparse=T)
testY = test_data[,153]
data = list(x = testX, y = testY )
dtest = xgb.DMatrix(data = data$x, label = data$y)

Warning message:
"package 'xgboost' was built under R version 4.2.3"


After preparing the training data and testing data, we start to train the XGBoost model， we tried different combinations of hyper-parameters. Lastly, we can get the best model below.

In [4]:
xgb <- xgboost(data = dtrain,max_depth=4,eta=0.1, nround=246)
#test data (oos r2)
pre_xgb = predict(xgb,newdata = dtest)
rss = var(testY - pre_xgb)
tss = var(testY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_xgb = predict(xgb,newdata = dtrain)
rss = var(trainY - pre_xgb)
tss = var(trainY)
is_r2 = 1-rss/tss
is_r2

[1]	train-rmse:92.061983 
[2]	train-rmse:82.914972 
[3]	train-rmse:74.677771 
[4]	train-rmse:67.259995 
[5]	train-rmse:60.580245 
[6]	train-rmse:54.565229 
[7]	train-rmse:49.148945 
[8]	train-rmse:44.271963 
[9]	train-rmse:39.880766 
[10]	train-rmse:35.927167 
[11]	train-rmse:32.367789 
[12]	train-rmse:29.163569 
[13]	train-rmse:26.279350 
[14]	train-rmse:23.683477 
[15]	train-rmse:21.347461 
[16]	train-rmse:19.245668 
[17]	train-rmse:17.353170 
[18]	train-rmse:15.648035 
[19]	train-rmse:14.111950 
[20]	train-rmse:12.727826 
[21]	train-rmse:11.481129 
[22]	train-rmse:10.356435 
[23]	train-rmse:9.342795 
[24]	train-rmse:8.429963 
[25]	train-rmse:7.606032 
[26]	train-rmse:6.863698 
[27]	train-rmse:6.194104 
[28]	train-rmse:5.591272 
[29]	train-rmse:5.046713 
[30]	train-rmse:4.555711 
[31]	train-rmse:4.113265 
[32]	train-rmse:3.713992 
[33]	train-rmse:3.354419 
[34]	train-rmse:3.029902 
[35]	train-rmse:2.737759 
[36]	train-rmse:2.473634 
[37]	train-rmse:2.235456 
[38]	train-rmse:2.021250 

[1] 0.9890334

[1] 0.9999999

Then we use random forest.

In [5]:
library(randomForest)

rf = randomForest(CPI ~ . ,ntree=100, data = train_data)
#test data (oos r2)
pre_rf = predict(rf,newdata = test_data)
rss = var(testY - pre_rf)
tss = var(testY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_rf = predict(rf,newdata = train_data)
rss = var(trainY - pre_rf)
tss = var(trainY)
is_r2 = 1-rss/tss
is_r2

randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.



[1] 0.8240949

[1] 0.9968749

kNN

In [6]:
library(caret)
control = trainControl(method = 'cv',number = 10)
kNN = train(CPI~.,train_data,
               method = 'knn',
               trControl = control,
               tuneLength = 5)
#test data (oos r2)
pre_kNN = predict(kNN,newdata = test_data)
rss = var(testY - pre_kNN)
tss = var(testY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_kNN = predict(kNN,newdata = train_data)
rss = var(trainY - pre_kNN)
tss = var(trainY)
is_r2 = 1-rss/tss
is_r2

Warning message:
"package 'caret' was built under R version 4.2.3"
Loading required package: ggplot2


Attaching package: 'ggplot2'


The following object is masked from 'package:randomForest':

    margin


Loading required package: lattice



[1] -0.8655443

[1] 0.8042666

decision tree

In [7]:
library(rpart)
DT = rpart(CPI~., train_data)
#test data (oos r2)
pre_DT = predict(DT, newdata = test_data)
rss = var(testY - pre_DT)
tss = var(testY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_DT = predict(DT, newdata = train_data)
rss = var(trainY - pre_DT)
tss = var(trainY)
is_r2 = 1-rss/tss
is_r2

[1] 0.7363672

[1] 0.9525064

lasso

In [8]:
library(glmnet)
x = trainX
y = trainY
cv_lasso = cv.glmnet(x, y)
lasso_result = glmnet(x, y, lambda = cv_lasso$lambda.min)
#test data (oos r2)
pre_lasso = predict(lasso_result, newx = testX)
rss = var(testY - pre_lasso)
tss = var(testY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_lasso = predict(lasso_result, newx = trainX)
rss = var(trainY - pre_lasso)
tss = var(trainY)
is_r2 = 1-rss/tss
is_r2

Loaded glmnet 4.1-6



,s0
s0,0.9991502


,s0
s0,0.9991502


neural network

In [9]:
library(neuralnet)
nn = neuralnet(CPI ~ ., train_data,hidden = 5)
#test data (oos r2)
pre_nn = predict(nn, newdata = test_data)
rss = var(testY - pre_nn)
tss = var(testY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_nn = predict(nn, newdata = train_data)
rss = var(trainY - pre_nn)
tss = var(trainY)
is_r2 = 1-rss/tss
is_r2

Warning message:
"package 'neuralnet' was built under R version 4.2.3"


0


0


By comparing the results of CPI above, we can see that the xgboost has the best r2 in the train data set, but lasso has the best r2 in the test data set.

As for ppi, we first use XGBoost.

In [10]:
PPItrainX = data.matrix(train_data[,c(2:153)])
PPItrainX = Matrix(trainX,sparse=T)
PPItrainY = train_data[,154]
data = list(x = PPItrainX, y = PPItrainY )
PPIdtrain = xgb.DMatrix(data = data$x, label = data$y)

PPItestX = data.matrix(test_data[,c(2:153)])
PPItestX = Matrix(testX,sparse=T)
PPItestY = test_data[,154]
data = list(x = PPItestX, y = PPItestY )
PPIdtest = xgb.DMatrix(data = data$x, label = data$y)

In [11]:
PPIxgb <- xgboost(data = PPIdtrain, max_depth = 12, eta = 0.3,nround=63)
#test data (oos r2)
pre_PPIxgb = predict(PPIxgb,newdata = PPIdtest)
rss = var(PPItestY - pre_PPIxgb)
tss = var(PPItestY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_PPIxgb = predict(PPIxgb,newdata = PPIdtrain)
rss = var(PPItrainY - pre_PPIxgb)
tss = var(PPItrainY)
is_r2 = 1-rss/tss
is_r2

[1]	train-rmse:70.993197 
[2]	train-rmse:49.922646 
[3]	train-rmse:35.150519 
[4]	train-rmse:24.770549 
[5]	train-rmse:17.472204 
[6]	train-rmse:12.334498 
[7]	train-rmse:8.718835 
[8]	train-rmse:6.173712 
[9]	train-rmse:4.382402 
[10]	train-rmse:3.116947 
[11]	train-rmse:2.227998 
[12]	train-rmse:1.602847 
[13]	train-rmse:1.163920 
[14]	train-rmse:0.854254 
[15]	train-rmse:0.632544 
[16]	train-rmse:0.473067 
[17]	train-rmse:0.357687 
[18]	train-rmse:0.272066 
[19]	train-rmse:0.209777 
[20]	train-rmse:0.162117 
[21]	train-rmse:0.126232 
[22]	train-rmse:0.098904 
[23]	train-rmse:0.078215 
[24]	train-rmse:0.062221 
[25]	train-rmse:0.049936 
[26]	train-rmse:0.040409 
[27]	train-rmse:0.032898 
[28]	train-rmse:0.026999 
[29]	train-rmse:0.022311 
[30]	train-rmse:0.018596 
[31]	train-rmse:0.015604 
[32]	train-rmse:0.013116 
[33]	train-rmse:0.011061 
[34]	train-rmse:0.009347 
[35]	train-rmse:0.007862 
[36]	train-rmse:0.006659 
[37]	train-rmse:0.005616 
[38]	train-rmse:0.004766 
[39]	train-rmse

[1] 0.9619968

[1] 1

randomForest

In [12]:
rf = randomForest(PPI ~ . , data = train_data,ntree = 500)

#test data (oos r2)
pre_rf = predict(rf,newdata = test_data)
rss = var(PPItestY - pre_rf)
tss = var(PPItestY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_rf = predict(rf,newdata = train_data)
rss = var(PPItrainY - pre_rf)
tss = var(PPItrainY)
is_r2 = 1-rss/tss
is_r2

[1] 0.9887479

[1] 0.9990204

kNN

In [13]:
control = trainControl(method = 'cv',number = 10)
kNN = train(PPI~.,train_data,
               method = 'knn',
               trControl = control,
               tuneLength = 5)
#test data (oos r2)
pre_kNN = predict(kNN,newdata = test_data)
rss = var(PPItestY - pre_kNN)
tss = var(PPItestY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_kNN = predict(kNN,newdata = train_data)
rss = var(PPItrainY - pre_kNN)
tss = var(PPItrainY)
is_r2 = 1-rss/tss
is_r2

[1] 0.463413

[1] 0.885504

decision tree

In [14]:
DT = rpart(PPI~., train_data)
#test data (oos r2)
pre_DT = predict(DT, newdata = test_data)
rss = var(PPItestY - pre_DT)
tss = var(PPItestY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_DT = predict(DT, newdata = train_data)
rss = var(PPItrainY - pre_DT)
tss = var(PPItrainY)
is_r2 = 1-rss/tss
is_r2

[1] 0.7936427

[1] 0.9568245

lasso

In [15]:
x = PPItrainX
y = PPItrainY
cv_lasso = cv.glmnet(x, y)
lasso_result = glmnet(x, y, lambda = cv_lasso$lambda.min)
#test data (oos r2)
pre_lasso = predict(lasso_result, newx = PPItestX)
rss = var(PPItestY - pre_lasso)
tss = var(PPItestY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_lasso = predict(lasso_result, newx = PPItrainX)
rss = var(PPItrainY - pre_lasso)
tss = var(PPItrainY)
is_r2 = 1-rss/tss
is_r2

,s0
s0,0.9991502


,s0
s0,0.9991502


neural network

In [16]:
nn = neuralnet(PPI ~ ., train_data,hidden = 20)
#test data (oos r2)
pre_nn = predict(nn, newdata = test_data)
rss = var(PPItestY - pre_nn)
tss = var(PPItestY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_nn = predict(nn, newdata = train_data)
rss = var(PPItrainY - pre_nn)
tss = var(PPItrainY)
is_r2 = 1-rss/tss
is_r2

0


0.007932918


By comparing the results of PPI above, we can see that the xgboost has the best r2 in the train data set, but lasso has the best r2 in the test data set.

We can now use lasso to make the prediction of both CPI and PPI.